In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from collections import Counter
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/suicide-watch/Suicide_Detection.csv


In [3]:
df=pd.read_csv("/kaggle/input/suicide-watch/Suicide_Detection.csv")
df.head()

,Unnamed: 0,text,class
0,2,Ex Wife Threatening SuicideRecently I left my ...,suicide
1,3,Am I weird I don't get affected by compliments...,non-suicide
2,4,Finally 2020 is almost over... So I can never ...,non-suicide
3,8,i need helpjust help me im crying so hard,suicide
4,9,"I’m so lostHello, my name is Adam (16) and I’v...",suicide


In [4]:
length=df['text'].apply(lambda x: len(x.split()))

In [5]:
new_df=df[length.values > 150]
new_df

,Unnamed: 0,text,class
4,9,"I’m so lostHello, my name is Adam (16) and I’v...",suicide
5,11,Honetly idkI dont know what im even doing here...,suicide
6,12,[Trigger warning] Excuse for self inflicted bu...,suicide
8,16,"Everyone wants to be ""edgy"" and it's making me...",non-suicide
9,18,My life is over at 20 years oldHello all. I am...,suicide
...,...,...,...
232056,348081,No hopeI contacted the suicide prevention hotl...,suicide
232060,348087,"I'm going to try again. Please, SW, you may be...",suicide
232062,348092,paralyzing anxiety &amp; depression are pushin...,suicide
232067,348100,I’m a fucking scummy person and I can’t change...,suicide


In [6]:
length[(length.values >= 150)]

4          438
5          291
6          243
8          195
9          224
          ... 
232056     248
232060     154
232062    1627
232067     165
232072     364
Name: text, Length: 60020, dtype: int64

In [7]:
for index, text in enumerate(df["text"]):
    words = text.split()
    num_words = len(words)

    if 10 <= num_words <= 150:
    
        continue
    else:
        
        df.at[index, "text"] = ""

# Remove rows with empty text
df = df[df["text"] != ""]

# Optional: Reset index if needed
df.reset_index(drop=True, inplace=True)

In [8]:
new_df['text'][4]

'I’m so lostHello, my name is Adam (16) and I’ve been struggling for years and I’m afraid. Through these past years thoughts of suicide, fear, anxiety I’m so close to my limit . I’ve been quiet for so long and I’m too scared to come out to my family about these feelings. About 3 years ago  losing my aunt triggered it all. Everyday feeling hopeless , lost, guilty, and remorseful over her and all the things I’ve done in my life,but thoughts like these with the little I’ve experienced in life? Only time I’ve revealed these feelings to my family is when I broke down where they saw my cuts. Watching them get so worried over something I portrayed as an average day made me feel absolutely dreadful. They later found out I was an attempt survivor from attempt OD(overdose from pills) and attempt hanging. All that happened was a blackout from the pills and I never went through with the noose because I’m still so afraid. During my first therapy I was diagnosed with severe depression, social anxiet

In [9]:
new_df['class'] 

4             suicide
5             suicide
6             suicide
8         non-suicide
9             suicide
             ...     
232056        suicide
232060        suicide
232062        suicide
232067        suicide
232072        suicide
Name: class, Length: 59663, dtype: object

In [10]:
df.drop(columns='Unnamed: 0',inplace=True)
df.head()

,text,class
0,Ex Wife Threatening SuicideRecently I left my ...,suicide
1,Am I weird I don't get affected by compliments...,non-suicide
2,Finally 2020 is almost over... So I can never ...,non-suicide
3,I took the rest of my sleeping pills and my pa...,suicide
4,Can you imagine getting old? Me neither.Wrinkl...,suicide


## lowercase

In [11]:
df["clean_text"]=df["text"].str.lower()
df.head()

,text,class,clean_text
0,Ex Wife Threatening SuicideRecently I left my ...,suicide,ex wife threatening suiciderecently i left my ...
1,Am I weird I don't get affected by compliments...,non-suicide,am i weird i don't get affected by compliments...
2,Finally 2020 is almost over... So I can never ...,non-suicide,finally 2020 is almost over... so i can never ...
3,I took the rest of my sleeping pills and my pa...,suicide,i took the rest of my sleeping pills and my pa...
4,Can you imagine getting old? Me neither.Wrinkl...,suicide,can you imagine getting old? me neither.wrinkl...


## Removal pantecuation

In [12]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [13]:
def remove_punctuation(text):
    punctuation=string.punctuation
    return text.translate(str.maketrans('','',punctuation))

In [14]:
df["clean_text"]=df["clean_text"].apply(lambda x:remove_punctuation(x))
df.head()

,text,class,clean_text
0,Ex Wife Threatening SuicideRecently I left my ...,suicide,ex wife threatening suiciderecently i left my ...
1,Am I weird I don't get affected by compliments...,non-suicide,am i weird i dont get affected by compliments ...
2,Finally 2020 is almost over... So I can never ...,non-suicide,finally 2020 is almost over so i can never hea...
3,I took the rest of my sleeping pills and my pa...,suicide,i took the rest of my sleeping pills and my pa...
4,Can you imagine getting old? Me neither.Wrinkl...,suicide,can you imagine getting old me neitherwrinkles...


## Removal Stopwords

In [15]:
','.join(stopwords.words("english"))

"i,me,my,myself,we,our,ours,ourselves,you,you're,you've,you'll,you'd,your,yours,yourself,yourselves,he,him,his,himself,she,she's,her,hers,herself,it,it's,its,itself,they,them,their,theirs,themselves,what,which,who,whom,this,that,that'll,these,those,am,is,are,was,were,be,been,being,have,has,had,having,do,does,did,doing,a,an,the,and,but,if,or,because,as,until,while,of,at,by,for,with,about,against,between,into,through,during,before,after,above,below,to,from,up,down,in,out,on,off,over,under,again,further,then,once,here,there,when,where,why,how,all,any,both,each,few,more,most,other,some,such,no,nor,not,only,own,same,so,than,too,very,s,t,can,will,just,don,don't,should,should've,now,d,ll,m,o,re,ve,y,ain,aren,aren't,couldn,couldn't,didn,didn't,doesn,doesn't,hadn,hadn't,hasn,hasn't,haven,haven't,isn,isn't,ma,mightn,mightn't,mustn,mustn't,needn,needn't,shan,shan't,shouldn,shouldn't,wasn,wasn't,weren,weren't,won,won't,wouldn,wouldn't"

In [16]:
STOP_words=set(stopwords.words("english"))
def remove_stop(text):
    return " ".join(word for word in text.split() if word not in STOP_words)

In [17]:
new_df["clean_text"]=df["clean_text"].apply(lambda x:remove_stop(x))
new_df.head()

,Unnamed: 0,text,class,clean_text
4,9,"I’m so lostHello, my name is Adam (16) and I’v...",suicide,imagine getting old neitherwrinkles weight gai...
5,11,Honetly idkI dont know what im even doing here...,suicide,think getting hit train would painfulguns hard...
6,12,[Trigger warning] Excuse for self inflicted bu...,suicide,death continuedi posted saw something interest...
8,16,"Everyone wants to be ""edgy"" and it's making me...",non-suicide,well im screwed locked school toilet cant get ...
9,18,My life is over at 20 years oldHello all. I am...,suicide,im fucked assignment due tomorrow havent even ...


In [18]:
print(new_df["text"][4])
new_df["clean_text"][4]

I’m so lostHello, my name is Adam (16) and I’ve been struggling for years and I’m afraid. Through these past years thoughts of suicide, fear, anxiety I’m so close to my limit . I’ve been quiet for so long and I’m too scared to come out to my family about these feelings. About 3 years ago  losing my aunt triggered it all. Everyday feeling hopeless , lost, guilty, and remorseful over her and all the things I’ve done in my life,but thoughts like these with the little I’ve experienced in life? Only time I’ve revealed these feelings to my family is when I broke down where they saw my cuts. Watching them get so worried over something I portrayed as an average day made me feel absolutely dreadful. They later found out I was an attempt survivor from attempt OD(overdose from pills) and attempt hanging. All that happened was a blackout from the pills and I never went through with the noose because I’m still so afraid. During my first therapy I was diagnosed with severe depression, social anxiety

'imagine getting old neitherwrinkles weight gain hair loss messed teeth bones health issues menopause hormones hating new generations amp way world progress useless angry piece shit cant take care totally depended people secretly wants die already even imagine absolutely even happy id take life avoid'

## discover more frequent words

In [26]:
new_df['clean_text'].isnull()

4         False
5         False
6         False
8         False
9         False
          ...  
232056     True
232060     True
232062     True
232067     True
232072     True
Name: clean_text, Length: 59663, dtype: bool

In [20]:
word_count=Counter()
for text in new_df["clean_text"]:
    for word in text:
        word_count[word]+=1
word_count.most_common(10)
        

TypeError: 'float' object is not iterable

In [51]:
def remove_im(text):
    return " ".join(word for word in text.split() if word != 'im' )

In [52]:
new_df["clean_text"]=new_df["clean_text"].apply(lambda x:remove_im(x))
df.head()

/tmp/ipykernel_43/2923437815.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["clean_text"]=new_df["clean_text"].apply(lambda x:remove_im(x))


,Unnamed: 0,text,class
0,2,Ex Wife Threatening SuicideRecently I left my ...,suicide
1,3,Am I weird I don't get affected by compliments...,non-suicide
2,4,Finally 2020 is almost over... So I can never ...,non-suicide
3,19,I took the rest of my sleeping pills and my pa...,suicide
4,20,Can you imagine getting old? Me neither.Wrinkl...,suicide


In [16]:
print(df["text"][3])
df["clean_text"][3]

I took the rest of my sleeping pills and my painkillersI can’t wait for it to end, I’ve struggled for the past 6 years and I’m finally ending it.


'took rest sleeping pills painkillersi can’t wait end i’ve struggled past 6 years i’m finally ending'

In [53]:
word_count.most_common()[:-10:-1]

[('conk', 1),
 ('passovermy', 1),
 ('postlexapro', 1),
 ('disorderelements', 1),
 ('ednosbinge', 1),
 ('16march', 1),
 ('asshatted', 1),
 ('beliieve', 1),
 ('tonightwhich', 1)]

## remove special characters

In [8]:

def remove_spl_chars(text):
    text=re.sub('[^a-zA-Z0-9]',' ',text)
    text=re.sub('\s+',' ',text)
    return text

In [10]:
df["clean_text"]=df["clean_text"].apply(lambda x:remove_spl_chars(x))
df.tail()

,Unnamed: 0,text,class,clean_text
232069,348103,If you don't like rock then your not going to ...,non-suicide,if you dont like rock then your not going to g...
232070,348106,You how you can tell i have so many friends an...,non-suicide,you how you can tell i have so many friends an...
232071,348107,pee probably tastes like salty tea😏💦‼️ can som...,non-suicide,pee probably tastes like salty tea can someone...
232072,348108,The usual stuff you find hereI'm not posting t...,suicide,the usual stuff you find hereim not posting th...
232073,348110,I still haven't beaten the first boss in Hollo...,non-suicide,i still havent beaten the first boss in hollow...


In [20]:
print(df["text"][150000])
df["clean_text"][150000]

Hey guys it’s 4 am and I wanna answer really deeeeeeeep questions about shit. Like... the deepest philosophical questions you got.  I’ve got the itch again.


'hey guys it s 4 wanna answer really deeeeeeeep questions shit like deepest philosophical questions got i ve got itch'

In [21]:
df.tail(10)

,text,class,clean_text
164469,I’ve fucking had enough.None of this depressio...,suicide,i ve fucking enoughnone depression subs helped...
164470,i have a cute ribbon tied around my neck and m...,non-suicide,cute ribbon tied around neck thighs soft hell ...
164471,How ba-a-ad can i be? I’m just doin’ what come...,non-suicide,baaad i m doin comes naturally baaad song make...
164472,I know no one here cares but anyways I was fig...,non-suicide,know one cares anyways fighting sans undertale...
164473,My mom tried to kill herself today. I feel los...,suicide,mom tried kill today feel lostits second time ...
164474,I did something today I went sledding with my ...,non-suicide,something today went sledding friends may seem...
164475,If you don't like rock then your not going to ...,non-suicide,dont like rock going get anything go httpsmusi...
164476,You how you can tell i have so many friends an...,non-suicide,tell many friends lonely everything deprived p...
164477,pee probably tastes like salty tea😏💦‼️ can som...,non-suicide,pee probably tastes like salty tea someone dra...
164478,I still haven't beaten the first boss in Hollo...,non-suicide,still havent beaten first boss hollow knight i...


## removal url

In [11]:
def remove_url(text):
    return re.sub(r'\bhttp\w*\b','',text)
    

In [12]:
df["clean_text"]=df["clean_text"].apply(lambda x:remove_url(x))


In [24]:
df["clean_text"][150000]

'hey guys it s 4 wanna answer really deeeeeeeep questions shit like deepest philosophical questions got i ve got itch'

## Removal html tags

In [13]:
def remove_html_tags(text):
    return re.sub(r'<.*?>',' ',text)

In [14]:
df["clean_text"]=df["clean_text"].apply(lambda x:remove_url(x))
df.head()

,Unnamed: 0,text,class,clean_text
0,2,Ex Wife Threatening SuicideRecently I left my ...,suicide,ex wife threatening suiciderecently i left my ...
1,3,Am I weird I don't get affected by compliments...,non-suicide,am i weird i dont get affected by compliments ...
2,4,Finally 2020 is almost over... So I can never ...,non-suicide,finally 2020 is almost over so i can never hea...
3,8,i need helpjust help me im crying so hard,suicide,i need helpjust help me im crying so hard
4,9,"I’m so lostHello, my name is Adam (16) and I’v...",suicide,i m so losthello my name is adam 16 and i ve b...


## Lemmatization 

In [15]:
import spacy 

# Load the spaCy English language model
nlp = spacy.load("en_core_web_sm")
# Define a counter variable
call_counter = 0

# Define a function for lemmatization
def lemmatize_text(text):
    global call_counter  # Use the global keyword to access the counter variable

    call_counter += 1  # Increment the counter
    doc = nlp(text)
    lemmatized_text = " ".join([token.lemma_ for token in doc])

      # Check if the counter is a multiple of 10,000
    if call_counter % 10000 == 0:
        print(f"Reached {call_counter} function calls.")

    return lemmatized_text

In [16]:
df["clean_text"]=df["clean_text"].apply(lambda x:lemmatize_text(x))
df.tail()

Reached 10000 function calls.
Reached 20000 function calls.
Reached 30000 function calls.
Reached 40000 function calls.
Reached 50000 function calls.
Reached 60000 function calls.
Reached 70000 function calls.
Reached 80000 function calls.
Reached 90000 function calls.
Reached 100000 function calls.
Reached 110000 function calls.
Reached 120000 function calls.
Reached 130000 function calls.
Reached 140000 function calls.
Reached 150000 function calls.
Reached 160000 function calls.
Reached 170000 function calls.
Reached 180000 function calls.
Reached 190000 function calls.
Reached 200000 function calls.
Reached 210000 function calls.
Reached 220000 function calls.
Reached 230000 function calls.


,Unnamed: 0,text,class,clean_text
232069,348103,If you don't like rock then your not going to ...,non-suicide,if you do not like rock then your not go to ge...
232070,348106,You how you can tell i have so many friends an...,non-suicide,you how you can tell I have so many friend and...
232071,348107,pee probably tastes like salty tea😏💦‼️ can som...,non-suicide,pee probably taste like salty tea can someone ...
232072,348108,The usual stuff you find hereI'm not posting t...,suicide,the usual stuff you find hereim not post this ...
232073,348110,I still haven't beaten the first boss in Hollo...,non-suicide,I still have not beat the first boss in hollow...


In [69]:
new_df.head()

,Unnamed: 0,text,class,clean_text
4,9,"I’m so lostHello, my name is Adam (16) and I’v...",suicide,I m losthello name adam 16 I ve struggle year ...
5,11,Honetly idkI dont know what im even doing here...,suicide,honetly idki do not know even feel like nothin...
6,12,[Trigger warning] Excuse for self inflicted bu...,suicide,trigger warn excuse self inflict burnsi know c...
8,16,"Everyone wants to be ""edgy"" and it's making me...",non-suicide,everyone want edgy make self conscious feel li...
9,18,My life is over at 20 years oldHello all. I am...,suicide,life 20 year oldhello 20 year old bald male ha...


In [ ]:
print(df["clean_text"][0])
df["text"][0]

In [33]:
import time
# Measure time taken
start_time = time.time()

# Call the function
result = correct_spellings("realy i lve you")
# Calculate elapsed
elapsed_time = time.time() - start_time

print("Original Text:", "Ex Wife Threatening SuicideRecently I left my wife for good because she has cheated on me twice and lied to me so much that I have decided to refuse to go back to her. As of a few days ago, she began threatening suicide. I have tirelessly spent these paat few days talking her out of it and she keeps hesitating because she wants to believe I'll come back. I know a lot of people will threaten this in order to get their way, but what happens if she really does? What do I do and how am I supposed to handle her death on my hands? I still love my wife but I cannot deal with getting cheated on again and constantly feeling insecure. I'm worried today may be the day she does it and I hope so much it doesn't happen.")
print("Result:", result)
print(f"Time taken: {elapsed_time:.6f} seconds")

Original Text: Ex Wife Threatening SuicideRecently I left my wife for good because she has cheated on me twice and lied to me so much that I have decided to refuse to go back to her. As of a few days ago, she began threatening suicide. I have tirelessly spent these paat few days talking her out of it and she keeps hesitating because she wants to believe I'll come back. I know a lot of people will threaten this in order to get their way, but what happens if she really does? What do I do and how am I supposed to handle her death on my hands? I still love my wife but I cannot deal with getting cheated on again and constantly feeling insecure. I'm worried today may be the day she does it and I hope so much it doesn't happen.
Result: really i love you
Time taken: 0.001253 seconds


In [19]:
df.drop(columns='Unnamed: 0',inplace=True)

In [20]:
# Save DataFrame to CSV
df.to_csv('nice_clean.csv', index=False)

# Download the CSV file
from IPython.display import FileLink
FileLink(r'nice_clean.csv')

/kaggle/working/nice_clean.csv